In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
import pcntoolkit as ptk
from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix
from pcntoolkit.util.utils import calibration_descriptives
from pcntoolkit.model.hbr import bspline_fit, bspline_transform
from sklearn.preprocessing import OneHotEncoder

In [ ]:
projdir = '/project_cephfs/3022017.02/projects/stijdboe/make_results/10_folds_results/'
textfiles = os.path.join(projdir,'textfiles')
if not os.path.exists(textfiles):
    os.mkdir(textfiles)
data_dir = '/project_cephfs/3022017.02/projects/stijdboe/Data'
folds_dir = os.path.join(data_dir,'10_folds_sexcov')
python_path = '/project_cephfs/3022017.02/projects/stijdboe/stijn2/bin/python3.10'
normative_path = '/project_cephfs/3022017.02/projects/stijdboe/PCNtoolkit/pcntoolkit/normative.py'

### Fit the models

Now we fit the models. This involves looping over the IDPs we have selected. We will use a module from PCNtoolkit to set up the design matrices, containing the covariates, fixed effects for site and nonlinear basis expansion. 

In [ ]:
warp =  'WarpSinArcsinh'
# For each fold
for i in range(10):

    this_identifier = f"fold_{i}_blr"
    fold_dir = os.path.join(folds_dir,f'fold_{i}')
    processing_dir = os.path.join(projdir, this_identifier+'/')
    if not os.path.exists(processing_dir):
        os.mkdir(processing_dir)

    log_dir = os.path.join(processing_dir, 'log')           #
    if not os.path.isdir(log_dir):
        os.mkdir(log_dir)

    # The paths to the data
    X_tr_path = os.path.join(fold_dir, 'X_train.pkl')
    Y_tr_path = os.path.join(fold_dir, 'Y_train.pkl')
    Z_tr_path = os.path.join(fold_dir, 'Z_train.pkl')
    
    X_te_path = os.path.join(fold_dir, 'X_test.pkl')
    Y_te_path = os.path.join(fold_dir, 'Y_test.pkl')
    Z_te_path = os.path.join(fold_dir, 'Z_test.pkl')
    
    with open(X_tr_path, 'rb') as file:
        X_tr = pickle.load(file)
    with open(Z_tr_path, 'rb') as file:
        Z_tr = pickle.load(file)
    with open(X_te_path, 'rb') as file:
        X_te = pickle.load(file)
    with open(Z_te_path, 'rb') as file:
        Z_te = pickle.load(file)
    print(Z_tr)
    
    
    
    Phi_tr = create_design_matrix(X_tr, site_ids = Z_tr['site_id'].to_numpy(), basis='bspline')
    Phi_cov_tr = create_design_matrix(X_tr, basis='bspline')
    Phi_te = create_design_matrix(X_te, site_ids = Z_te['site_id'].to_numpy(), basis='bspline')
    Phi_cov_te = create_design_matrix(X_te, basis='bspline')

    # Save as text files 
    Phi_tr_path = os.path.join(textfiles, f'Phi_tr_{i}.txt' )
    Phi_te_path = os.path.join(textfiles, f'Phi_te_{i}.txt' )
    np.savetxt(Phi_tr_path, Phi_tr)
    np.savetxt(Phi_te_path, Phi_te)
    
    Phi_cov_tr_path = os.path.join(textfiles, f'Phi_cov_tr_{i}.txt' )
    Phi_cov_te_path = os.path.join(textfiles, f'Phi_cov_te_{i}.txt' )
    np.savetxt(Phi_cov_tr_path, Phi_cov_tr)
    np.savetxt(Phi_cov_te_path, Phi_cov_te)
    
    
    # Load response variables
    with open(Y_tr_path, 'rb') as file:
        Y_tr = pickle.load(file)
    with open(Y_te_path, 'rb') as file:
        Y_te = pickle.load(file)
    
    # Loop over features
    for iff, feature in enumerate(Y_tr.columns):
        batch_dir = os.path.join(processing_dir, f'batch_{iff+1}')
        if not os.path.exists(batch_dir):
            os.mkdir(batch_dir)
        os.chdir(batch_dir)
        
        F_tr = Y_tr[feature].to_numpy()
        F_tr_mean = np.mean(F_tr, axis = 0)
        F_tr_sd = np.std(F_tr, axis = 0)
        F_tr = (F_tr-F_tr_mean)/F_tr_sd
        F_te = Y_te[feature].to_numpy()
        F_te = (F_te-F_tr_mean)/F_tr_sd
        
        # Save as text files 
        Y_feature_tr_path = os.path.join(textfiles, f'Y_{feature}_tr_{i}.txt' )
        Y_feature_te_path = os.path.join(textfiles, f'Y_{feature}_te_{i}.txt' )
        np.savetxt(Y_feature_tr_path, F_tr)
        np.savetxt(Y_feature_te_path, F_te)
        
#         # Fit the model
        estimate(
                 Phi_tr_path,  
                 Y_feature_tr_path,
                 testcov=Phi_te_path, 
                 testresp=Y_feature_te_path,
                 alg='blr', optimizer = 'powell', 
                 savemodel=True, warp=warp, warp_reparam=True)


